## 自建一個cmrc2018的2筆資料

In [5]:
source_json = [{
    'id':'lesson1',
    'context':'小英的生日是1997年10月9日,女性',
    'question':'小英的生日是?',
    'answer':{'text':['1997年10月9日'],'answer_start':[6]}
    },
    {
    'id':'lesson2',
    'context':'川普日前宣布課徵加拿大和墨西哥25%的關稅',
    'question':'加拿大和墨西哥被課徵的關稅是',
    'answer':{'text':['25%'],'answer_start':[15]}
    }

]

## 轉換為DatasetDict

In [10]:
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_list(source_json)
datasets = DatasetDict({
    'train':train_dataset
})
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answer'],
        num_rows: 2
    })
})

# 目標
![](./images/pic1.png)

## 使用AutoTokenizer建立input_ids,token_type_ids,attention_mask

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [22]:
tokenized_dataset = tokenizer(
    text=datasets['train']['question'],
    text_pair=datasets['train']['context'],
    max_length = 512, #BERT最高處理512byte,
    truncation="only_second", #全部超過只截斷context,
    padding=True
)
#查資訊類型
print(type(tokenized_dataset))
#取出第一筆
print(tokenized_dataset['input_ids'][0])
print(tokenized_dataset['token_type_ids'][0])
print(tokenized_dataset['attention_mask'][0])

<class 'transformers.tokenization_utils_base.BatchEncoding'>
[101, 2207, 5739, 4638, 4495, 3189, 3221, 136, 102, 2207, 5739, 4638, 4495, 3189, 3221, 8387, 2399, 8108, 3299, 130, 3189, 117, 1957, 2595, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
